# MySQL Data Analysis Assignments

## MySQL Python Connection Basics

In [1]:
## importing 'mysql.connector' as mysql for convenient
import mysql.connector as mysql
import pandas as pd
import openpyxl

Dictionary of parameters to connect to MySQL database. 

Connecting to the database using 'connect()' method takes 3 required parameters `host`, `user`, `password / passwd`

In [2]:
config = {
    'host':'localhost',
    'user':'root',
    'password':'root',
}

Connect to MySQL database

In [3]:
db = mysql.connect(**config)

Creating an instance of 'cursor' class which is used to execute the 'SQL' statements in 'Python'

In [4]:
cursor = db.cursor(buffered=True)

In [5]:
# assign mysql query to a variable
query = '''SHOW DATABASES'''

`execute()` method is used to compile a 'SQL' statement

In [6]:
cursor.execute(query)

- `fetchall()` method fetches all the rows from the last executed statement.
- Use only when query returns some output, if query doesn't return any output then fetchall will throw an error `No result set to fecth from.`

In [7]:
databases = cursor.fetchall()
print(databases)

[('employees',), ('information_schema',), ('mavenfuzzyfactory',), ('mysql',), ('performance_schema',), ('sakila',), ('sql_hr',), ('sql_inventory',), ('sql_invoicing',), ('sql_store',), ('sys',), ('world',)]


Use `for loop` to loop throgh the results if they are in list or tuple format

In [8]:
for i in databases:
    print(i[0])

employees
information_schema
mavenfuzzyfactory
mysql
performance_schema
sakila
sql_hr
sql_inventory
sql_invoicing
sql_store
sys
world


- Here is a custom function which return values if the query has `show` or `select` keywords. If it doesn't then it just executes the query.
- You can also select the from vaious `fetch` methods in case of query returns some output. Generally there are 3 fetch methods.
1. `fetchone()` - only returns one row/record
2. `fetchmany(many)` - returns multiple rows/records depending parameter `many` which takes input
3. `fetchall()` - returns all the rows/records

In [9]:
def query_execute(query, fetch_type=3, fetch_many=10):
    substr = ['show', 'select']

    if any(x in query.lower() for x in substr):

        if isinstance(fetch_type, int) == True:

            if isinstance(fetch_many, int) == True:

                if fetch_type == 1:
                    qexecute = cursor.execute(query)
                    qfetch = cursor.fetchone()
                    return [i for i in qfetch]

                elif fetch_type == 2:
                    qexecute = cursor.execute(query)
                    qfetch = cursor.fetchmany(fetch_many)
                    return [i for i in qfetch]

                elif fetch_type == 3:
                    qexecute = cursor.execute(query)
                    qfetch = cursor.fetchall()
                    return [i for i in qfetch]

                else:
                    error =  "Enter these fetch_type - '1' for fetchone(), '2' for fetchmany() and 3 for fetchall()"
                    return error 

            else:
                return 'fetch_many is {}, it should be integer value'.format(type(fetch_many).__name__)                    

        else:
            return 'fetch_type is {}, it should be integer value'.format(type(fetch_type).__name__)

    else:
        qexecute = cursor.execute(query)        
        return qexecute

Select and use `mavenfuzzyfactory` for the analysis 

In [10]:
query_execute('''USE mavenfuzzyfactory''')

List out table names in the `mavenfuzzyfactory` database

In [11]:
query_execute('''SHOW TABLES''')

[('order_item_refunds',),
 ('order_items',),
 ('orders',),
 ('products',),
 ('website_pageviews',),
 ('website_sessions',)]

Print first 2 rows from the orders table.

In [12]:
query_execute('''SELECT * FROM website_sessions''', fetch_type=2, fetch_many=2)

[(1,
  datetime.datetime(2012, 3, 19, 8, 4, 16),
  1,
  0,
  'gsearch',
  'nonbrand',
  'g_ad_1',
  'mobile',
  'https://www.gsearch.com'),
 (2,
  datetime.datetime(2012, 3, 19, 8, 16, 49),
  2,
  0,
  'gsearch',
  'nonbrand',
  'g_ad_1',
  'desktop',
  'https://www.gsearch.com')]

### **Important**

- Fetch method for fetching data from MySQL server to python for analysis is not suitableas it returns list of tuples where values in tuplers are rows for each column.

- Another better alternative is to use `read_sql()` function in `Pandas` and read SQL data into pandas dataframe. Most important paramters are `sq' for sql statement and `con` for connection to MySQL server.
- More: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql.html


Read `orders` table and return as dataframe.

In [13]:
sql = 'SELECT * FROM orders'
df_orders = pd.read_sql(sql=sql, con=db)
df_orders.head()

,order_id,created_at,website_session_id,user_id,primary_product_id,items_purchased,price_usd,cogs_usd
0,1,2012-03-19 10:42:46,20,20,1,1,49.99,19.49
1,2,2012-03-19 19:27:37,104,104,1,1,49.99,19.49
2,3,2012-03-20 06:44:45,147,147,1,1,49.99,19.49
3,4,2012-03-20 09:41:45,160,160,1,1,49.99,19.49
4,5,2012-03-20 11:28:15,177,177,1,1,49.99,19.49


In [14]:
# check the shape of dataframe
df_orders.shape

(32313, 8)

Create orders summary by product id

In [15]:
filter_cols = ['primary_product_id', 'user_id', 'items_purchased', 'price_usd']
agg_cols = {'user_id': 'count',
            'items_purchased':'sum',
            'price_usd': 'sum'
           }
df_orders_price_filter = df_orders[filter_cols].groupby('primary_product_id', as_index=False).agg(agg_cols)

In [16]:
df_orders_price_filter

,primary_product_id,user_id,items_purchased,price_usd
0,1,23861,29618,1419767.82
1,2,4803,5682,318109.18
2,3,3068,4097,180857.03
3,4,581,628,19775.72


Save summary data to excel or csv

In [17]:
# with pd.ExcelWriter('products_orders_summary.xlsx', 
#                     options={'encoding':'utf-8', 'engine':'openpyxl'}) as writer:
#     df_orders_price_filter.to_excel(writer, index=False, sheet_name='orders_summary')

In [18]:
df_orders_price_filter.to_csv('products_orders_summary.csv', index=False, encoding='utf-8')